In [1]:

from __future__ import print_function, division
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.nn import init
import torch.nn.functional as F
import functools
import numpy as np
import torchvision
from torchvision import transforms, datasets, models
import os
import time

import Augmentor

from torchsummary import summary

from tensorboardX import SummaryWriter

import matplotlib.pyplot as plt

In [2]:
# Training settings
batch_size = 50

In [70]:
#%%

# p = Augmentor.Pipeline()
# p.random_erasing(0.5,0.4)




train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop((32, 32), padding=4),   #left, top, right, bottom
#     p.torch_transform(),
#     p.sample(10000),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                               train=True,
                               transform=train_transform,
                               download=True)

test_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                              train=False,
                              transform=test_transform)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, # 64
                                           shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

dataloaders = {
    'train': train_loader,
    'test' : test_loader    
}
image_datasets = {
    'train': train_dataset,
    'test' : test_dataset
}
dataset_sizes = {
    'train': len(image_datasets['train']),
    'test' : len(image_datasets['test'])
}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print('train_size: ',dataset_sizes['train'],'\ntest_size : ',dataset_sizes['test'])
print(classes)
print('len class: ', len(classes))



Files already downloaded and verified
train_size:  50000 
test_size :  10000
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
len class:  10


In [160]:
import cv2
# # ADDING NOISE
noise = []
noise = np.random.rand(32, 32,3)
import pickle

pickle_out = open("batch1","wb")
data = []
filenames = []
for n in range(1):
    inputs, labels = next(iter(dataloaders['train']))

    for m in range(batch_size):
        images_ = inputs[m].numpy()
        class_  = labels[m]
        images_ = np.transpose(images_, [1,2,0])
        x_random = np.random.randint(6, 27, size=1, dtype='uint8')
        y_random = np.random.randint(6, 27, size=1, dtype='uint8')
        w_random = np.random.randint(5, 10, size=1, dtype='uint8')
        h_random = np.random.randint(5, 10, size=1, dtype='uint8')
        images_[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]=noise[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]

        r = images_[:,:,0]
        g = images_[:,:,1]
        b = images_[:,:,2]

        images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
        images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
        images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
        data.append(images_)
        filename = '{}_{}.png'.format(classes[class_.numpy()],n*m+m)
        filenames.append(filename)
#         print(classes[class_.numpy()])
        plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occulde/{}_{}.png'.format(classes[class_.numpy()],n*m+m), images_)
#         plt.imshow(images_)
#         plt.show()

    pickle_cifar10 = {'labels': labels.numpy(),'data': data, 'filenames': filenames}
    pickle.dump(pickle_cifar10, pickle_out)
    pickle_out.close()

#         images_[:,:,0] = ((r - r.min()) / (r.max() - r.min()) * 255).astype('uint8')
#         images_[:,:,1] = ((g - g.min()) / (g.max() - g.min()) * 255).astype('uint8')
#         images_[:,:,2] = ((b - b.min()) / (b.max() - b.min()) * 255).astype('uint8')


In [138]:
def unpickle(file):
#     import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [161]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch1")
print(pickle_in)

{'labels': array([1, 8, 8, 8, 8, 9, 0, 3, 4, 6, 5, 5, 6, 4, 1, 7, 8, 1, 2, 2, 0, 5,
       3, 3, 1, 6, 0, 1, 5, 1, 7, 2, 7, 9, 8, 2, 7, 0, 6, 4, 7, 4, 3, 0,
       9, 4, 7, 3, 3, 3]), 'data': [array([[[1.        , 1.        , 1.        ],
        [0.992126  , 0.992126  , 0.9920635 ],
        [0.992126  , 0.992126  , 0.9920635 ],
        ...,
        [0.992126  , 0.992126  , 0.9920635 ],
        [0.992126  , 0.992126  , 0.9920635 ],
        [1.        , 1.        , 1.        ]],

       [[1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       [[1.        , 1.        , 1.        ],
        [0.996063  , 0.996063  , 0.99603176],
        [0.996063  , 0.996063  , 0.99603176],
        ...,
        [0.996063  , 0.996063  , 0.99603176],
        [0.996063  , 0.99606

In [142]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/data/cifar-10-batches-py/data_batch_1")
print(pickle_in)

{b'batch_label': b'training batch 1 of 5', b'labels': [6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7, 1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1, 4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7, 9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9, 9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7, 6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4, 6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4, 1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3, 5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9, 0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3, 6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4, 0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5, 6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2, 1, 1, 2, 1, 5, 9, 9, 0, 8, 4, 1, 1, 6, 3, 